In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [6]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [7]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
79555,Grant_Hanley,21,2,90,Norwich,Crystal Palace,0.283219,0.461424,True,1,...,1,1,6.0,1152,6558,5406,1,2020-01-01T17:30:00Z,1920,1.0
79556,Roberto_Pereyra,21,3,1,Watford,Wolverhampton Wanderers,0.471687,0.744747,True,1,...,1,2,0.0,-910,266,1176,0,2020-01-01T15:00:00Z,1920,1.0
79557,Sead_Kolasinac,21,2,68,Arsenal,Manchester United,1.428641,1.551383,True,5,...,0,2,6.0,-1322,16,1338,1,2020-01-01T20:00:00Z,1920,0.0
79558,Stuart_Armstrong,21,3,89,Southampton,Tottenham Hotspur,0.470650,2.007002,True,3,...,0,1,26.0,-95,134,229,0,2020-01-01T15:00:00Z,1920,1.0
79559,Rúben Diogo_da Silva Neves,21,3,36,Wolverhampton Wanderers,Watford,0.744747,0.471687,False,1,...,1,2,17.0,-3555,7427,10982,0,2020-01-01T15:00:00Z,1920,1.0


In [8]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

22

In [9]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [11]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Ederson_Santana de Moraes,22,1,90,Manchester City,Aston Villa,2.697171,0.534040,False,2,...,865688,6,1,0.0,-29148,6716,35864,0,2020-01-12T16:30:00Z,1920
1,Xherdan_Shaqiri,22,3,1,Liverpool,Tottenham Hotspur,2.468949,2.027028,False,1,...,17519,1,0,0.0,-610,18,628,0,2020-01-11T17:30:00Z,1920
2,Hugo_Lloris,22,1,0,Tottenham Hotspur,Liverpool,2.027028,2.468949,True,0,...,239378,1,0,0.0,-3102,81,3183,0,2020-01-11T17:30:00Z,1920
3,Cenk_Tosun,22,4,22,Crystal Palace,Arsenal,0.482482,1.416533,True,1,...,8407,1,1,4.0,281,137,27,0,2020-01-11T12:30:00Z,1920
4,Callum_Robinson,22,3,0,Sheffield United,West Ham United,0.243160,0.704895,True,0,...,19466,0,1,0.0,-526,173,699,0,2020-01-10T20:00:00Z,1920
5,Heung-Min_Son,22,3,90,Tottenham Hotspur,Liverpool,2.027028,2.468949,True,2,...,455973,1,0,30.0,48462,84236,35774,0,2020-01-11T17:30:00Z,1920
6,João Filipe Iria_Santos Moutinho,22,3,90,Wolverhampton Wanderers,Newcastle United,0.703754,0.559829,True,6,...,407903,1,1,0.0,10429,32832,22403,0,2020-01-11T15:00:00Z,1920
7,Andrew_Robertson,22,2,90,Liverpool,Tottenham Hotspur,2.468949,2.027028,False,6,...,1413313,1,0,10.0,44946,80176,35230,0,2020-01-11T17:30:00Z,1920
8,Danny_Rose,22,2,68,Tottenham Hotspur,Liverpool,2.027028,2.468949,True,2,...,94624,1,0,0.0,-4696,18,4714,0,2020-01-11T17:30:00Z,1920
9,Martin_Kelly,22,2,90,Crystal Palace,Arsenal,0.482482,1.416533,True,2,...,2306467,1,1,0.0,-64516,33817,98333,0,2020-01-11T12:30:00Z,1920


In [12]:
df_latest.shape

(597, 34)

In [13]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [14]:
last_play_proba.head()

,player,play_proba
31,Ahmed_El Mohamady,1.0
32,James_Chester,1.0
33,Neil_Taylor,1.0
34,Kortney_Hause,1.0
35,Jonathan_Kodjia,1.0


In [15]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [16]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
0,Ederson_Santana de Moraes,22,1,90,Manchester City,Aston Villa,2.697171,0.534040,False,2,...,6,1,0.0,-29148,6716,35864,0,2020-01-12T16:30:00Z,1920,0.75
1,Xherdan_Shaqiri,22,3,1,Liverpool,Tottenham Hotspur,2.468949,2.027028,False,1,...,1,0,0.0,-610,18,628,0,2020-01-11T17:30:00Z,1920,0.00
2,Hugo_Lloris,22,1,0,Tottenham Hotspur,Liverpool,2.027028,2.468949,True,0,...,1,0,0.0,-3102,81,3183,0,2020-01-11T17:30:00Z,1920,0.00
3,Cenk_Tosun,22,4,22,Crystal Palace,Arsenal,0.482482,1.416533,True,1,...,1,1,4.0,281,137,27,0,2020-01-11T12:30:00Z,1920,1.00
4,Callum_Robinson,22,3,0,Sheffield United,West Ham United,0.243160,0.704895,True,0,...,0,1,0.0,-526,173,699,0,2020-01-10T20:00:00Z,1920,1.00
5,Heung-Min_Son,22,3,90,Tottenham Hotspur,Liverpool,2.027028,2.468949,True,2,...,1,0,30.0,48462,84236,35774,0,2020-01-11T17:30:00Z,1920,1.00
6,João Filipe Iria_Santos Moutinho,22,3,90,Wolverhampton Wanderers,Newcastle United,0.703754,0.559829,True,6,...,1,1,0.0,10429,32832,22403,0,2020-01-11T15:00:00Z,1920,1.00
7,Andrew_Robertson,22,2,90,Liverpool,Tottenham Hotspur,2.468949,2.027028,False,6,...,1,0,10.0,44946,80176,35230,0,2020-01-11T17:30:00Z,1920,1.00
8,Danny_Rose,22,2,68,Tottenham Hotspur,Liverpool,2.027028,2.468949,True,2,...,1,0,0.0,-4696,18,4714,0,2020-01-11T17:30:00Z,1920,0.00
9,Martin_Kelly,22,2,90,Crystal Palace,Arsenal,0.482482,1.416533,True,2,...,1,1,0.0,-64516,33817,98333,0,2020-01-11T12:30:00Z,1920,1.00


In [17]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [18]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
80152,Grant_Hanley,22,2,79,Norwich,Manchester United,0.316088,1.546205,False,0,...,0,4,0.0,2918,9581,6663,0,2020-01-11T15:00:00Z,1920,1.0
80153,Roberto_Pereyra,22,3,8,Watford,Bournemouth,0.488603,0.713194,False,6,...,3,0,17.0,-745,372,1117,0,2020-01-12T14:00:00Z,1920,1.0
80154,Sead_Kolasinac,22,2,90,Arsenal,Crystal Palace,1.416533,0.482482,False,2,...,1,1,6.0,4606,5911,1305,0,2020-01-11T12:30:00Z,1920,1.0
80155,Stuart_Armstrong,22,3,77,Southampton,Leicester City,0.470968,1.048266,False,9,...,2,1,25.0,13,184,171,1,2020-01-11T15:00:00Z,1920,1.0
80156,Rúben Diogo_da Silva Neves,22,3,90,Wolverhampton Wanderers,Newcastle United,0.703754,0.559829,True,2,...,1,1,3.0,-7613,5844,13457,0,2020-01-11T15:00:00Z,1920,1.0


In [19]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [20]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [21]:
fixtures.head(10)

,home_team,away_team,gw
179,West Ham United,Liverpool,24
220,Arsenal,Sheffield United,23
221,Brighton and Hove Albion,Aston Villa,23
222,Burnley,Leicester City,23
223,Liverpool,Manchester United,23
224,Manchester City,Crystal Palace,23
225,Newcastle United,Chelsea,23
226,Norwich,Bournemouth,23
227,Southampton,Wolverhampton Wanderers,23
228,Watford,Tottenham Hotspur,23


In [22]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [23]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,24,West Ham United,Liverpool,21,14
1,23,Arsenal,Sheffield United,3,49
2,23,Brighton and Hove Albion,Aston Villa,36,7
3,23,Burnley,Leicester City,90,13
4,23,Liverpool,Manchester United,14,1


In [24]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [25]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [26]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,24,Liverpool,West Ham United,Andrew_Robertson,2,70,100,False
1,24,Liverpool,West Ham United,Trent_Alexander-Arnold,2,75,100,False
2,24,Liverpool,West Ham United,Virgil_van Dijk,2,64,100,False
3,24,Liverpool,West Ham United,Joseph_Gomez,2,52,100,False
4,24,Liverpool,West Ham United,Joel_Matip,2,52,50,False


In [27]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,24,West Ham United,Liverpool,Aaron_Cresswell,2,47,100,True
1,24,West Ham United,Liverpool,Angelo_Ogbonna,2,45,100,True
2,24,West Ham United,Liverpool,Arthur_Masuaku,2,42,100,True
3,24,West Ham United,Liverpool,Pablo_Zabaleta,2,42,None,True
4,24,West Ham United,Liverpool,Ryan_Fredericks,2,45,0,True


In [28]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 28.0,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 12.5,
 'Aaron_Mooy': 66.0,
 'Aaron_Ramsdale': 72.0,
 'Aaron_Wan-Bissaka': 72.0,
 'Abdoulaye_Doucouré': 83.25,
 'Adam_Idah': 13.0,
 'Adam_Lallana': 26.25,
 'Adam_Masina': 28.0,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 78.4,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 81.5,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 54.0,
 'Ainsley_Maitland-Niles': 90.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 5.25,
 'Alex_Iwobi': 10.0,
 'Alex_McCarthy': 90.0,
 'Alexander_Tettey': 85.0,
 'Alexandre_Lacazette': 72.6,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 33.2,
 'Alisson_Ramses Becker': 90.0,
 'Andre_Gray': 2.2,
 'Andreas_Christensen': 18.0,
 'Andreas_Pereira': 52.8,
 'Andrew_Robertson': 89.25,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 0.0,
 'Andy_Carroll': 38.2,
 'Andy_Lonergan': 0.0,
 'Angel_Gomes': 2.0,
 'Angelo_Ogbonna': 67.5,
 'Angus_Gunn': 0.0,
 'Anthony_Driscoll-Glennon': 0.0,
 'Anthon

In [29]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [30]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,24,West Ham United,Liverpool,Aaron_Cresswell,2,4.7,1.00,True,0.704895,2.468949,1920,90.000000
1,24,West Ham United,Liverpool,Angelo_Ogbonna,2,4.5,1.00,True,0.704895,2.468949,1920,67.500000
2,24,West Ham United,Liverpool,Arthur_Masuaku,2,4.2,1.00,True,0.704895,2.468949,1920,41.250000
3,24,West Ham United,Liverpool,Pablo_Zabaleta,2,4.2,1.00,True,0.704895,2.468949,1920,45.000000
4,24,West Ham United,Liverpool,Ryan_Fredericks,2,4.5,0.00,True,0.704895,2.468949,1920,0.000000
5,24,West Ham United,Liverpool,Issa_Diop,2,4.4,1.00,True,0.704895,2.468949,1920,22.500000
6,24,West Ham United,Liverpool,Fabián_Balbuena,2,4.3,1.00,True,0.704895,2.468949,1920,90.000000
7,24,West Ham United,Liverpool,Winston_Reid,2,3.9,1.00,True,0.704895,2.468949,1920,0.000000
8,24,West Ham United,Liverpool,Ben_Johnson,2,3.9,1.00,True,0.704895,2.468949,1920,0.000000
9,24,West Ham United,Liverpool,Marko_Arnautovic,4,7.0,0.00,True,0.704895,2.468949,1920,0.000000


In [31]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')